In [13]:
library(tidyverse)
library(gprofiler2)

In [2]:
dir.create('wgcna_output/renamed_tables', recursive = TRUE, showWarnings = FALSE)

In [8]:
kme = read_csv('wgcna_output/tables/dmb_ds4_mcs20_pF_kME_cellModEmbed.csv.gz')
kme %>% head

Rows: 144 Columns: 386
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (3): data, cell_cluster, cell_id
dbl (383): HPF__bisque2, HPF__blanchedalmond, HPF__blue4, HPF__cadetblue4, H...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


data,cell_cluster,cell_id,HPF__bisque2,HPF__blanchedalmond,HPF__blue4,HPF__cadetblue4,HPF__chocolate,HPF__cornflowerblue,HPF__cyan3,⋯,ZI__royalblue3,ZI__salmon3,ZI__sienna,ZI__steelblue1,ZI__thistle1,ZI__thistle2,ZI__thistle3,ZI__thistle4,ZI__tomato4,ZI__yellow3
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dmb,HY,S0010_01_chow_HY,0.05197933,0.03023810,0.1824172,0.1229980,-0.05775205,0.019439584,-0.06125762,⋯,-0.08595527,0.05252190,-0.05822008,0.06567769,0.036785185,0.0102144634,0.10537119,0.11220870,-0.04791984,0.020875004
dmb,HY,S0010_02_chow_HY,0.08876762,0.08017922,0.1459963,0.1170462,-0.03252780,-0.004745673,-0.05191100,⋯,-0.11281907,0.02927096,-0.02412367,0.02603893,0.007287426,-0.0017344272,0.10817653,0.11961064,-0.04316019,0.006892997
dmb,HY,S0010_03_chow_HY,0.07109426,0.02853630,0.1832191,0.1384336,-0.06336754,-0.032457115,-0.06025051,⋯,-0.10252465,0.06811403,-0.04506138,0.01064958,0.021634418,-0.0002412614,0.07374108,0.10598614,-0.02872247,0.025098740
dmb,HY,S0010_04_chow_HY,0.02948085,0.06371331,0.1535754,0.1107229,-0.03764149,0.025134355,-0.05945944,⋯,-0.05751049,0.03972461,-0.01765908,0.05303416,-0.004217487,0.0049177488,0.06565093,0.01816232,-0.03737867,0.014236332
dmb,HY,S0010_05_chow_HY,0.11411558,0.03807216,0.1615871,0.1345847,-0.06702112,0.007702094,-0.05492529,⋯,-0.15425467,0.03670603,-0.01827709,0.01769293,-0.020943932,-0.0073797921,0.09252465,0.09507013,-0.02601267,0.019362943
dmb,HY,S0010_06_chow_HY,0.02021261,0.07189122,0.1562818,0.1019575,-0.03508683,-0.025868732,-0.07505565,⋯,-0.09517898,0.05157318,-0.02545144,0.02872648,-0.025390663,0.0255168968,0.07634631,0.05995981,-0.04782112,0.008990742


In [9]:
modules = colnames(kme) %>%
`[`(-(1:3)) %>%
enframe %>%
rename(index = name, old_name = value) %>%
separate(old_name, into = c('region', 'module'), sep = '__', remove = FALSE) %>%
group_by(region) %>%
mutate(row_number = row_number()) %>%
mutate(row_number = str_pad(as.character(row_number), width=2, pad="0",side = 'left')) %>% 
mutate(new_name = paste0(region, '__', row_number))

modules %>% head
modules %>% write_csv('wgcna_output/renamed_tables/module_mappings.csv')

index,old_name,region,module,row_number,new_name
<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,HPF__bisque2,HPF,bisque2,01,HPF__01
2,HPF__blanchedalmond,HPF,blanchedalmond,02,HPF__02
3,HPF__blue4,HPF,blue4,03,HPF__03
4,HPF__cadetblue4,HPF,cadetblue4,04,HPF__04
5,HPF__chocolate,HPF,chocolate,05,HPF__05
6,HPF__cornflowerblue,HPF,cornflowerblue,06,HPF__06


In [10]:
new_colnames = c(colnames(kme)[(1:3)], modules$new_name)
colnames(kme) = new_colnames

kme = kme %>%
rename(region = cell_cluster,
       sample = cell_id)

kme %>% head
kme %>% write_csv('wgcna_output/renamed_tables/kME_cellModEmbed.csv')

data,region,sample,HPF__01,HPF__02,HPF__03,HPF__04,HPF__05,HPF__06,HPF__07,⋯,ZI__39,ZI__40,ZI__41,ZI__42,ZI__43,ZI__44,ZI__45,ZI__46,ZI__47,ZI__48
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dmb,HY,S0010_01_chow_HY,0.05197933,0.03023810,0.1824172,0.1229980,-0.05775205,0.019439584,-0.06125762,⋯,-0.08595527,0.05252190,-0.05822008,0.06567769,0.036785185,0.0102144634,0.10537119,0.11220870,-0.04791984,0.020875004
dmb,HY,S0010_02_chow_HY,0.08876762,0.08017922,0.1459963,0.1170462,-0.03252780,-0.004745673,-0.05191100,⋯,-0.11281907,0.02927096,-0.02412367,0.02603893,0.007287426,-0.0017344272,0.10817653,0.11961064,-0.04316019,0.006892997
dmb,HY,S0010_03_chow_HY,0.07109426,0.02853630,0.1832191,0.1384336,-0.06336754,-0.032457115,-0.06025051,⋯,-0.10252465,0.06811403,-0.04506138,0.01064958,0.021634418,-0.0002412614,0.07374108,0.10598614,-0.02872247,0.025098740
dmb,HY,S0010_04_chow_HY,0.02948085,0.06371331,0.1535754,0.1107229,-0.03764149,0.025134355,-0.05945944,⋯,-0.05751049,0.03972461,-0.01765908,0.05303416,-0.004217487,0.0049177488,0.06565093,0.01816232,-0.03737867,0.014236332
dmb,HY,S0010_05_chow_HY,0.11411558,0.03807216,0.1615871,0.1345847,-0.06702112,0.007702094,-0.05492529,⋯,-0.15425467,0.03670603,-0.01827709,0.01769293,-0.020943932,-0.0073797921,0.09252465,0.09507013,-0.02601267,0.019362943
dmb,HY,S0010_06_chow_HY,0.02021261,0.07189122,0.1562818,0.1019575,-0.03508683,-0.025868732,-0.07505565,⋯,-0.09517898,0.05157318,-0.02545144,0.02872648,-0.025390663,0.0255168968,0.07634631,0.05995981,-0.04782112,0.008990742


In [31]:
me = read_csv('wgcna_output/tables/dmb_ds4_mcs20_pF_geneMod.csv.gz')

me = me %>%
mutate(old_name = paste0(cell_cluster, '__', module)) %>%
left_join(modules %>% select(old_name, new_name), 
          by = 'old_name') %>%
select(-region, -old_name) %>%
rename(name = new_name) %>%
relocate(name, .after='module')

mouse_ensgs = me %>% pull(genes)
#
gprofiler_result <- gorth(
    query = mouse_ensgs,
    source_organism = 'mmusculus',
    target_organism = 'hsapiens',
    filter_na = FALSE
) %>%
distinct(input, .keep_all=TRUE) %>%
filter(ortholog_ensg != 'N/A') %>%
select(input, ortholog_ensg, ortholog_name) %>%
rename(genes = input)

me = me %>% 
left_join(gprofiler_result, by = 'genes') %>%
relocate(ortholog_ensg, ortholog_name, .after='genes')

me %>% head
me %>% write_csv('wgcna_output/renamed_tables/wgcna_output_ds4_mcs20_pF_geneMod.csv')

Rows: 19770 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): data, run, cell_cluster, module, genes
dbl (2): pkMs, gene_loadings

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Adding missing grouping variables: `region`


data,run,cell_cluster,module,name,genes,ortholog_ensg,ortholog_name,pkMs,gene_loadings
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
dmb,ds4_mcs20_pF,HPF,orangered,HPF__23,ENSMUSG00000120785,NA,NA,0.8369424,0.1501479
dmb,ds4_mcs20_pF,HPF,orangered,HPF__23,ENSMUSG00000108990,NA,NA,0.8353232,0.1498574
dmb,ds4_mcs20_pF,HPF,orangered,HPF__23,ENSMUSG00000110233,NA,NA,0.7805153,0.1400248
dmb,ds4_mcs20_pF,HPF,orangered,HPF__23,ENSMUSG00000083911,NA,NA,0.7797708,0.1398913
dmb,ds4_mcs20_pF,HPF,orangered,HPF__23,ENSMUSG00000112311,NA,NA,0.7761567,0.1392429
dmb,ds4_mcs20_pF,HPF,orangered,HPF__23,ENSMUSG00000103485,NA,NA,0.7539321,0.1352558
